In [16]:
#  Copyright 2016 The TensorFlow Authors. All Rights Reserved.
# Modified original from MNIST Example

from __future__ import absolute_import
from __future__ import division
from __future__ import print_function

import numpy as np
import tensorflow as tf
import cnn_data_batch as data
# tf.logging.set_verbosity(tf.logging.INFO)

Network parameters

In [17]:
# Parameters
learning_rate = 0.001
batch_size = 100

# Network Parameters
n_input = 1024 # data input
n_classes = 76 # total classes
dropout = 0.9 # Dropout, probability to keep units

# tf Graph input
x = tf.placeholder(tf.float32, [None, n_input])
y = tf.placeholder(tf.float32, [None, n_classes])
keep_prob = tf.placeholder(tf.float32) #dropout (keep probability)

In [18]:
# Create some wrappers for simplicity
def conv1d(x, W, b):
    # Conv2D wrapper, with bias and relu activation
    # [batch, height, width, channels]
    x = tf.nn.conv2d(x, W, [1,1,1,1], padding='SAME') #1d convolution
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool1d(x, k=4):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, 1, k, 1], strides=[1, 1, k, 1],
                          padding='SAME')

In [19]:
# Create model
def conv_net(x, weights, biases, do):
    # Reshape input picture
    x = tf.reshape(x, shape=[-1, 1, n_input, 1]) #-1 for batch size
    
    # First Layer: Convolution Layer
    conv1 = conv1d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool1d(conv1, k=4)

    # Second Layer: Convolution Layer
    conv2 = conv1d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool1d(conv2, k=4)
    print(conv2.shape)
    
    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    size = [-1, weights['wd1'].get_shape().as_list()[0]]
    print(size)
    fc1 = tf.reshape(conv2, size)
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, do)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [20]:
# Store layers weight & bias
weights = {
    # 1x4 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([1, 4, 1, 32])),
    # 1x4 conv, 32 inputs, 64 outputs
    'wc2': tf.Variable(tf.random_normal([1, 4, 32, 64])),
    # fully connected, 1*(1024/16)*64 inputs, 1024 outputs
    'wd1': tf.Variable(tf.random_normal([1*64*64, 64])),
    # Number inputs is channel depth, 76 outputs (class prediction)
    'out': tf.Variable(tf.random_normal([64, n_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([32])),
    'bc2': tf.Variable(tf.random_normal([64])),
    'bd1': tf.Variable(tf.random_normal([64])),
    'out': tf.Variable(tf.random_normal([n_classes]))
}

# Construct model
pred = conv_net(x, weights, biases, keep_prob)

# Define loss and optimizer
loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(loss)

# Evaluate model
correct_pred = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()

(?, 1, 64, 64)
[-1, 4096]


In [21]:
# Launch the graph
with tf.Session() as sess:
    sess.run(init)
    # Keep training until reach max iterations
    for step in range(2000):
        batch_x, batch_y = data.next_train_batch(batch_size)
        # Run optimization op (backprop)
        sess.run(optimizer, feed_dict={x: batch_x, y: batch_y, keep_prob: dropout})
        if step % 50 == 0:
            # Calculate batch loss and accuracy
            v_loss, v_acc = sess.run([loss, accuracy], feed_dict={x: batch_x,
                                                              y: batch_y,
                                                              keep_prob: 1.})
            print('accuracy=', v_acc, 'loss=', v_loss)
            
    print ("Optimization Finished!")

    # Calculate accuracy for 256 samples
    batch_x, batch_y = data.next_test_batch(1000)
    print ("Testing Accuracy:", \
        sess.run(accuracy, feed_dict={x: batch_x,
                                      y: batch_y, 
                                      keep_prob: 1.}))

ResourceExhaustedError: OOM when allocating tensor with shape[4096,1024]
	 [[Node: Variable_10/Adam_1/Assign = Assign[T=DT_FLOAT, _class=["loc:@Variable_10"], use_locking=true, validate_shape=true, _device="/job:localhost/replica:0/task:0/device:GPU:0"](Variable_10/Adam_1, Variable_10/Adam/Initializer/zeros)]]

Caused by op 'Variable_10/Adam_1/Assign', defined at:
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/home/ubuntu/anaconda3/envs/tensorflow_p36/lib/python